In [1]:
!pip install -q kaggle
!pip install transformers tokenizers datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.1 MB/s eta 0:00:00


In [2]:
from google.colab import files

## kaggle.json
files.upload()

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
!chmod 600 /content/kaggle.json
# ! kaggle datasets list

Saving kaggle.json to kaggle.json


In [3]:
!kaggle datasets download -d lotusacharya/nepalinewsdataset
!unzip -q nepalinewsdataset.zip

 55% 10.0M/18.1M [00:00<00:00, 40.5MB/s]
100% 18.1M/18.1M [00:00<00:00, 65.6MB/s]


In [4]:
path = '/content/'
news_class = ['ArthaBanijya/ArthaBanijya','Bichar/Bichar','Desh/Desh','Khelkud/Khelkud',
              'Manoranjan/Manoranjan','Prabas/Prabas','Sahitya/Sahitya',
              'SuchanaPrabidhi/SuchanaPrabidhi','Swasthya/Swasthya','Viswa/Viswa']

In [5]:
import os
import re
from transformers import set_seed
import random
import tensorflow as tf

# print("TESTS")
# print([os.path.exists(os.path.join(path,cls)) for cls in news_class])
# for cls in news_class:
#   print([os.path.exists(os.path.join(path,cls,"{}.txt".format(i+1))) for i in range(1000)])

###--------------------------

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

rand_seed = 9

def seed_everything(seed=0):
    random.seed(seed) # random
    os.environ['PYTHONHASHSEED'] = str(seed) # python enviroment
    np.random.seed(seed) # numpy
    tf.keras.utils.set_random_seed(seed) # tensorflow
    tf.random.set_seed(seed) # tensorflow
    set_seed(seed) # hugging_face transformer

seed_everything(rand_seed)

In [22]:
step = 80
data = pd.DataFrame(columns = ['label','text'])
for cls in news_class:

  label = cls.split("/")[0]
  print("Running",label)
  for i in tqdm(range(1000)):
    file = open(os.path.join(path,cls,"{}.txt".format(i+1)),'r',encoding='utf-8')
    text = file.read()
    text = re.sub('\s+'," ",text).strip().split(" ")
    text = [" ".join(text[j:j+step]) for j in range(0,len(text),step)]
    data = pd.concat([data,
                      pd.DataFrame({
                          'file':[int(i+1) for j in range(len(text))],
                          'label':[label for j in range(len(text))],
                          'text':text})])

data.index = range(data.shape[0])
print(data.shape)
data.head()

Running ArthaBanijya


  0%|          | 0/1000 [00:00<?, ?it/s]

Running Bichar


  0%|          | 0/1000 [00:00<?, ?it/s]

Running Desh


...

  0%|          | 0/1000 [00:00<?, ?it/s]

Running Viswa


  0%|          | 0/1000 [00:00<?, ?it/s]

(55322, 3)


,label,text,file
0,ArthaBanijya,"घर जग्गा कारोबारमा आत्मविश्वास गुमेको वर्ष""बैं...",1.0
1,ArthaBanijya,सर्वसाधारण पैसा राख्न डराउनु सेयरमा लगानी गरे ...,1.0
2,ArthaBanijya,बढेर माथि जान्छन् सबैभन्दा तल सूचक रहँदा सर्वस...,1.0
3,ArthaBanijya,र घरजग्गामा भएको लगानीमा जोखिम देखेपछि राष्ट्र...,1.0
4,ArthaBanijya,पहिलो कारोबारमा नेप्से परिसूचक सय बिन्दुमा थिय...,1.0


In [23]:
label2id = {news_class[i].split('/')[0]:i for i in range(len(news_class))}
id2label = {i:news_class[i].split('/')[0] for i in range(len(news_class))}

label2id, id2label

({'ArthaBanijya': 0,
  'Bichar': 1,
  'Desh': 2,
  'Khelkud': 3,
  'Manoranjan': 4,
  'Prabas': 5,
  'Sahitya': 6,
  'SuchanaPrabidhi': 7,
  'Swasthya': 8,
  'Viswa': 9},
 {0: 'ArthaBanijya',
  1: 'Bichar',
  2: 'Desh',
  3: 'Khelkud',
  4: 'Manoranjan',
  5: 'Prabas',
  6: 'Sahitya',
  7: 'SuchanaPrabidhi',
  8: 'Swasthya',
  9: 'Viswa'})

In [24]:
import datasets

data = datasets.Dataset.from_pandas(data).shuffle(999).train_test_split(test_size=0.2)
data


DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'file'],
        num_rows: 44257
    })
    test: Dataset({
        features: ['label', 'text', 'file'],
        num_rows: 11065
    })
})

In [25]:
def LabelEncoding(x):
    x['label'] = label2id[x['label']]
    return x

data = data.map(
        LabelEncoding,
        num_proc=4)

data['train'][:5]

Map (num_proc=4):   0%|          | 0/44257 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/11065 [00:00<?, ? examples/s]

{'label': [4, 5, 6, 1, 4],
 'text': ['सुनील र दिनेशलाई हरिताराचलचित्र क्षेत्रमा लामो समयदेखि पुर्\u200dयाएको योगदान बापत दिइने हरितारा चलचित्र पुरस्कार यस वर्ष कलाकार सुनील थापा र निर्देशक दिनेश डिसीलाई प्रदान गरिने भएको छ " चलचित्र क्षेत्रमा लामो समयदेखि पुर्याएको योगदान बापत दिइने हरितारा चलचित्र पुरस्कार यस वर्ष कलाकार सुनील थापा र निर्देशक दिनेश डिसीलाई प्रदान गरिने भएको छ हरितारा चलचित्र पुरस्कार गुठीद्वारा हरेक वर्ष दिइने पुरस्कारको राशि हजार रहेको छ मा स्थापित पुरस्कार यसअघि महजोडी शान्ति मास्के शिव श्रेष्ठ रवीन्द्र खड्का जयनन्द ठकुरी वसुन्धरा भुसाल शकुन्तला',
  'हो उनी स्वदेश फिर्ने तयारी गरेको अघिल्लो दिनदेखि बेपत्ता थिए बिगि्रएको गाडी राख्ने ठाउँमा फालिएको अवस्थामा प्रहरीले सेटेम्बर मा फेला पारेको हो उनले भने निधार र कपालको आधारमा शवको पहिचान भयो नेपाली दूतावासका द्वितीय सचिव जयबहादुर राईले घटना छानबिन भइरहेको जानकारी आएको बताए बुद्ध र खड्काको विषयमा छानबिन भइरहेको बुझिएको छ उनले भने बुद्ध दुई वर्षदेखि यहाँ टाइसर कम्पनीको सुरक्षा गार्ड थिए उनी कम्पनीको पोसाक लगाएको अवस्थामै म

In [26]:
print(tf.math.confusion_matrix(data['test']['label'],data['test']['label']))
print(tf.math.confusion_matrix(data['train']['label'],data['train']['label']))

tf.Tensor(
[[1104    0    0    0    0    0    0    0    0    0]
 [   0 2873    0    0    0    0    0    0    0    0]
 [   0    0  663    0    0    0    0    0    0    0]
 [   0    0    0  925    0    0    0    0    0    0]
 [   0    0    0    0  857    0    0    0    0    0]
 [   0    0    0    0    0  871    0    0    0    0]
 [   0    0    0    0    0    0  984    0    0    0]
 [   0    0    0    0    0    0    0  957    0    0]
 [   0    0    0    0    0    0    0    0 1120    0]
 [   0    0    0    0    0    0    0    0    0  711]], shape=(10, 10), dtype=int32)
tf.Tensor(
[[ 4491     0     0     0     0     0     0     0     0     0]
 [    0 11395     0     0     0     0     0     0     0     0]
 [    0     0  2660     0     0     0     0     0     0     0]
 [    0     0     0  3466     0     0     0     0     0     0]
 [    0     0     0     0  3515     0     0     0     0     0]
 [    0     0     0     0     0  3550     0     0     0     0]
 [    0     0     0     0     0     0  

In [27]:
from transformers import TFBertModel, BertTokenizerFast

model_id = 'Shushant/nepaliBERT'

tokenizer = BertTokenizerFast.from_pretrained(model_id)
tokenizer

model = TFBertModel.from_pretrained(model_id,
          id2label=id2label,
          label2id=label2id, from_pt=True)

model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['bert.embeddings.position_ids', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertModel were not initialized from the PyTorch model and are newly initialized: ['bert.pooler.dense.weight', 'bert.p

Model: "tf_bert_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
Total params: 109,482,240
Trainable params: 109,482,240
Non-trainable params: 0
_________________________________________________________________


### Code For Using BERT as Embedding Layer

In [28]:
import tensorflow as tf

class BERTEmbedding(tf.keras.layers.Layer):
  def __init__(self, model,tokenizer, max_token_len=128,padding='max_length',truncation=True):
    super().__init__()
    self.embedding = model
    self.tokenizer = tokenizer
    self.max_token = max_token_len
    self.padding = padding
    self.truncation = truncation
    self.trainable=False

  def call(self, x):
    return self.embedding(self.tokenizer(x,padding=self.padding,truncation=self.truncation,max_length=self.max_token,return_tensors='tf'))[1]

In [29]:
# model.bert(tokenizer('बैंक तथा वित्तीय संस्थामा देखिएको विश्वासको',padding='max_length',truncation=True,return_tensors='tf',max_length=128))
embd = BERTEmbedding(model.bert,tokenizer)
# embd(tokenizer(['बैंक तथा वित्तीय',' संस्थामा देखिएको विश्वासको'],padding='max_length',truncation=True,return_tensors='tf',max_length=128))
embd(["",'बैंक तथा वित्तीय संस्थामा देखिएको विश्वासको','बैंक तथा वित्तीय संस्थामा देखिएको विश्वासको'])

<tf.Tensor: shape=(3, 768), dtype=float32, numpy=
array([[-0.40543044,  0.4750434 , -0.23471402, ..., -0.5019045 ,
        -0.6270523 , -0.51111066],
       [-0.14002496,  0.61468726,  0.43341807, ...,  0.31418183,
        -0.64704937,  0.09423671],
       [-0.14002496,  0.61468726,  0.43341807, ...,  0.31418183,
        -0.64704937,  0.09423671]], dtype=float32)>

In [30]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data, batch_size=32,shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.data = data
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.data.num_rows / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.data.num_rows)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        chunk = self.data.select(indexes)
        X = embd(chunk['text'])
        y = np.array(chunk['label'])
        return X,y

In [31]:
%%time

train_features = DataGenerator(data['train'])
test_features = DataGenerator(data['test'])

CPU times: user 1.34 ms, sys: 3 µs, total: 1.34 ms
Wall time: 1.32 ms


## Model Specification

In [32]:
nc_model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(768)),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(10,activation='softmax')
])

In [33]:
nc_model.compile(
  optimizer=tf.keras.optimizers.Adam(
      learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(
              initial_learning_rate=0.0001,
              decay_steps=100000,
              decay_rate=0.95,
              staircase=True
          )
      ),
  loss='sparse_categorical_crossentropy',
  metrics=['acc'])

nc_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 128)               98432     
                                                                 
 dense_5 (Dense)             (None, 10)                1290      
                                                                 
Total params: 99,722
Trainable params: 99,722
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = nc_model.fit(train_features,
        epochs=30,
        validation_data=test_features,
        callbacks=[tf.keras.callbacks.EarlyStopping(
                            monitor='val_acc', patience=3,
                            verbose=1, mode='max',
                            restore_best_weights=True)
                        ])

Epoch 1/30
1384/1384 [==============================] - 503s 363ms/step - loss: 0.9020 - acc: 0.7175 - val_loss: 0.7191 - val_acc: 0.7616
Epoch 2/30
 904/1384 [==================>...........] - ETA: 2:20 - loss: 0.7038 - acc: 0.7666

## Evaluation Section

In [ ]:
%%time
from transformers import pipeline, TextClassificationPipeline

print("Getting Test Prediction")
if False:
  pred_labels = [np.argmax(tf.nn.softmax(model(tf.constant(x)).logits)) for x in tokenized_inputs['test']['input_ids']]
else:
  pipe = pipeline('sentiment-analysis',model=model,tokenizer=tokenizer,device=1)
  prediction = pipe(data['test']['text'])
  pred_labels = [model.config.label2id[x['label']] for x in prediction]

actual_labels = data['test']['label']

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

print("F1-Score",f1_score(actual_labels,pred_labels,average='weighted'))
print("Precision-Score",precision_score(actual_labels,pred_labels,average='weighted'))
print("Recall-Score",recall_score(actual_labels,pred_labels,average='weighted'))
print("accuracy_Score",accuracy_score(actual_labels,pred_labels))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cmd = ConfusionMatrixDisplay(tf.math.confusion_matrix(actual_labels,pred_labels,num_classes=3).numpy())
cmd.plot()